In [1]:
import pandas as pd
import requests

In [3]:
# init watcher. 
# WARNING! Actualise api_key every 24h
api_key = 'RGAPI-c8befa21-e65c-48c9-a658-0758455b8e05'

In [4]:
regions = [
    'br1', 
    'eun1', 
    'euw1', 
    'jp1', 
    'kr', 
    'la1', 
    'la2', 
    'na1', 
    'oc1', 
    'ph2', 
    'ru', 
    'sg2', 
    'th2', 
    'tr1', 
    'tw2', 
    'vn2'
]

In [5]:
df = pd.DataFrame(columns=[
                            'puuid', 'region', 'tier', 'placement', 'pair_id',
                            'champion_name', 'champion_rarity', 'champion_tier', 
                            'item_1', 'item_2', 'item_3', 
                            'augment_1', 'augment_2', 'augment_3'
                        ])

In [12]:
def accumulate_region_data(api_key, region, tier):
    players = []
    result = requests.get(f'https://{region}.api.riotgames.com/tft/league/v1/entries/{tier}/III?queue=RANKED_TFT&page=1', headers={"X-Riot-Token": api_key})
    while result.status_code != 200:
        result = requests.get(f'https://{region}.api.riotgames.com/tft/league/v1/entries/{tier}/III?queue=RANKED_TFT&page=1', headers={"X-Riot-Token": api_key})
        
    for player in result.json():
        player_info = {
            'summonerId': player['summonerId'],
            'summonerName': player['summonerName'],
            'region': region,  # Sprawdzamy, czy region jest poprawnie przypisany
            'tier': tier
        }
        players.append(player_info)
    return players

In [13]:
def add_puuid(api_key, players, region):
    for index, player in enumerate(players):
        result = requests.get(f'https://{player["region"]}.api.riotgames.com/tft/league/v1/entries/by-summoner/{player["summonerId"]}', headers={"X-Riot-Token": api_key})
        while result.status_code != 200:
            result = requests.get(f'https://{player["region"]}.api.riotgames.com/tft/league/v1/entries/by-summoner/{player["summonerId"]}', headers={"X-Riot-Token": api_key})
        players[index]['puuid'] = result.json()[0]['puuid']
    return players

In [14]:
import os

In [22]:
def save_matches(api_key, players, bigger_region, region, tier):
    for player in players:
        result = requests.get(f'https://{bigger_region}.api.riotgames.com/tft/match/v1/matches/by-puuid/{player["puuid"]}/ids?start=0&count=20', headers={"X-Riot-Token": api_key})
        while result.status_code != 200: 
            result = requests.get(f'https://{bigger_region}.api.riotgames.com/tft/match/v1/matches/by-puuid/{player["puuid"]}/ids?start=0&count=20', headers={"X-Riot-Token": api_key})

        
        for match in result.json():
            if f'{match}.csv' in os.listdir('matches'):
                continue
            
            result = requests.get(f'https://{bigger_region}.api.riotgames.com/tft/match/v1/matches/{match}', headers={"X-Riot-Token": api_key})
            while result.status_code != 200:
                result = requests.get(f'https://{bigger_region}.api.riotgames.com/tft/match/v1/matches/{match}', headers={"X-Riot-Token": api_key})

            if result.json()['info']['tft_game_type'] != 'standard' or result.json()['info']['tft_set_core_name'] !=  'TFTSet11':
                continue

            df = pd.DataFrame(columns=[
                            'puuid', 'bigger_region', 'region', 'tier', 'placement',
                            'champion_name', 'champion_rarity', 'champion_tier', 
                            'item_1', 'item_2', 'item_3', 
                            'augment_1', 'augment_2', 'augment_3'
                        ])

            
            for participant in result.json()['info']['participants']:
                while len(participant['augments']) < 3:
                    participant['augments'].append(pd.NA)
                for unit in participant['units']:
                    while len(unit['itemNames']) < 3:
                        unit['itemNames'].append(pd.NA)

                    df.loc[df.shape[0]] = [
                                            participant['puuid'], bigger_region, region, tier, participant['placement'], 
                                            unit['character_id'], unit['rarity'], unit['tier'],
                                            unit['itemNames'][0], unit['itemNames'][1], unit['itemNames'][2],
                                            participant['augments'][0], participant['augments'][1], participant['augments'][2]
                                          ]

            df.to_csv(f'matches/{match}.csv')
            print(f'{match}.csv is saved')
            print('=====================')

In [23]:
def get_tft_data(api_key, region, tier='challenger'):
    players = accumulate_region_data(api_key, region, tier)
    print(f'Accumulation of players in {tier} from {region} is done')
    players = add_puuid(api_key, players, region)
    print(f'Adding puuid to players is done.')
    regions_dict = {
                        'americas': ['br1', 'la1', 'la2', 'na1'],
                        'europe': ['eun1', 'euw1', 'ru'],
                        'asia': ['jp1', 'kr', 'ph2', 'sg2', 'th2', 'tr1', 'tw2', 'vn2'],
                        'sea': ['oc1']
                    }

    for key, value in zip(regions_dict.keys(), regions_dict.values()):
        if region in value:
            bigger_region = key
            save_matches(api_key, players, bigger_region, region, tier)

    print('Done')
    
    

In [24]:
get_tft_data(api_key, 'eun1', 'PLATINUM')

Accumulation of players in PLATINUM from eun1 is done
Adding puuid to players is done.
EUN1_3568803017.csv is saved
EUN1_3568785441.csv is saved
EUN1_3568755376.csv is saved
EUN1_3568739136.csv is saved
EUN1_3568703246.csv is saved
EUN1_3568679253.csv is saved
EUN1_3568634041.csv is saved
EUN1_3568604493.csv is saved
EUN1_3569050585.csv is saved
EUN1_3569007498.csv is saved
EUN1_3568965368.csv is saved
EUN1_3568866690.csv is saved
EUN1_3568860036.csv is saved
EUN1_3568863192.csv is saved
EUN1_3568841443.csv is saved
EUN1_3568573565.csv is saved
EUN1_3568542621.csv is saved
EUN1_3568508241.csv is saved
EUN1_3568471051.csv is saved
EUN1_3568467133.csv is saved
EUN1_3568446344.csv is saved
EUN1_3568440669.csv is saved
EUN1_3568421748.csv is saved
EUN1_3569007793.csv is saved
EUN1_3568985983.csv is saved
EUN1_3568971756.csv is saved
EUN1_3568951040.csv is saved
EUN1_3568947694.csv is saved
EUN1_3568943143.csv is saved
EUN1_3568929426.csv is saved
EUN1_3568911812.csv is saved
EUN1_356891817

In [ ]:
for region in regions:
    get_tft_data(api_key, region, 'PLATINUM')

Accumulation of players in PLATINUM from br1 is done
Adding puuid to players is done.
BR1_2910909465.csv is saved
BR1_2910895108.csv is saved
BR1_2910883191.csv is saved
BR1_2910868580.csv is saved
BR1_2910854396.csv is saved
BR1_2910848280.csv is saved
BR1_2910470171.csv is saved
BR1_2910458498.csv is saved
BR1_2910447048.csv is saved
BR1_2910430850.csv is saved
BR1_2910426091.csv is saved
BR1_2910412839.csv is saved
BR1_2910404241.csv is saved
BR1_2910395337.csv is saved
BR1_2910384668.csv is saved
BR1_2910383071.csv is saved
BR1_2910379640.csv is saved
BR1_2910909716.csv is saved
BR1_2910897729.csv is saved
BR1_2910871789.csv is saved
BR1_2910867251.csv is saved
BR1_2910849086.csv is saved
BR1_2910843101.csv is saved
BR1_2910831526.csv is saved
BR1_2910702498.csv is saved
BR1_2910683546.csv is saved
BR1_2910629217.csv is saved
BR1_2910606157.csv is saved
BR1_2910584316.csv is saved
BR1_2910559342.csv is saved
BR1_2910494183.csv is saved
BR1_2910431045.csv is saved
BR1_2910416579.csv